In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
import tensorflow as tf
# 查看gpu和cpu的数量
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [12]:
import sys
sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

np.random.seed(1234)
tf.set_random_seed(1234)

In [57]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x, y, z, t, u, v, w, layers):
        
        X = np.concatenate([x, y, z, t], 1)
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.z = X[:,2:3]
        self.t = X[:,3:4]
        
        self.u = u
        self.v = v
        self.w = w
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([0.001], dtype=tf.float32)
        self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.z_tf = tf.placeholder(tf.float32, shape=[None, self.z.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
         
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.v_tf = tf.placeholder(tf.float32, shape=[None, self.v.shape[1]])
        self.w_tf = tf.placeholder(tf.float32, shape=[None, self.w.shape[1]])
        
        self.u_pred, self.v_pred, self.w_pred, self.p_pred, self.f_u_pred, self.f_v_pred, self.f_w_pred, self.div = self.net_NS(self.x_tf, self.y_tf, self.z_tf, self.t_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_sum(tf.square(self.v_tf - self.v_pred)) + \
                    tf.reduce_sum(tf.square(self.w_tf - self.w_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u_pred)) + \
                    tf.reduce_sum(tf.square(self.f_v_pred)) + \
                    tf.reduce_sum(tf.square(self.f_w_pred)) + \
                    tf.reduce_sum(tf.square(self.div))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 5000,
                                                                           'maxfun': 5000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
        
    def net_NS(self, x, y, z, t):
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        
        output = self.neural_net(tf.concat([x,y,z,t], 1), self.weights, self.biases)
        u = output[:,0:1]
        v = output[:,1:2]
        w = output[:,2:3]
        p = output[:,3:4] 
        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_z = tf.gradients(u, z)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        u_zz = tf.gradients(u_z, z)[0]
        
        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_z = tf.gradients(v, z)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]
        v_zz = tf.gradients(v_z, z)[0]
        
        w_t = tf.gradients(w, t)[0]
        w_x = tf.gradients(w, x)[0]
        w_y = tf.gradients(w, y)[0]
        w_z = tf.gradients(w, z)[0]
        w_xx = tf.gradients(w_x, x)[0]
        w_yy = tf.gradients(w_y, y)[0]
        w_zz = tf.gradients(w_z, z)[0]
        
        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]
        p_z = tf.gradients(p, z)[0]

#         f_u = u_t + lambda_1*(u*u_x + v*u_y + w*u_z) + p_x - lambda_2*(u_xx + u_yy + u_zz) 
#         f_v = v_t + lambda_1*(u*v_x + v*v_y + w*v_z) + p_y - lambda_2*(v_xx + v_yy + v_zz)
#         f_w = w_t + lambda_1*(u*w_x + v*w_y + w*w_z) + p_z - lambda_2*(w_xx + w_yy + w_zz)
        f_u = u_t + (u*u_x + v*u_y + w*u_z) + lambda_1*p_x 
        f_v = v_t + (u*v_x + v*v_y + w*v_z) + lambda_1*p_y + 9.8
        f_w = w_t + (u*w_x + v*w_y + w*w_z) + lambda_1*p_z
        
        div = u_x + v_y + w_z
        
        return u, v, w, p, f_u, f_v, f_w, div
    
    #def callback(self, loss, lambda_1, lambda_2):
    #    print('Loss: %.3e, l1: %.3f, l2: %.5f' % (loss, lambda_1, lambda_2))
    def callback(self, loss):
        print('Loss: %.3e' % (loss))
      
    def train(self, nIter): 

        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.z_tf: self.z, self.t_tf: self.t, self.u_tf: self.u, self.v_tf: self.v, self.w_tf: self.w}
        
        # start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                # elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                #lambda_1_value = self.sess.run(self.lambda_1)
                # lambda_2_value = self.sess.run(self.lambda_2)
                # print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f, Time: %.2f' % 
                #      (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
                # print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f' % 
                #      (it, loss_value, lambda_1_value, lambda_2_value))
                print('It: %d, Loss: %.3e' % 
                      (it, loss_value))
                # start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
            
    
    def predict(self, x_star, y_star, z_star, t_star):
        
        tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.z_tf: z_star, self.t_tf: t_star}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        v_star = self.sess.run(self.v_pred, tf_dict)
        w_star = self.sess.run(self.w_pred, tf_dict)
        p_star = self.sess.run(self.p_pred, tf_dict)
        
        return u_star, v_star, w_star, p_star

In [58]:
N_train = 5000
layers = [4, 20, 20, 20, 20, 20, 20, 20, 20, 4]

# Load Data
position = scipy.io.loadmat('../Data/pos.mat')
velocity = scipy.io.loadmat('../Data/vel.mat')
time = scipy.io.loadmat('../Data/time.mat')

pos_star = position['pos'] # 99 x 1080 x 3
vel_star = velocity['vel'] # 99 x 1080 x 3
t_star = time['time']  # 1 x 99

N = pos_star.shape[1]
T = t_star.shape[1]

In [59]:
print(N)
print(T)

1080
99


In [60]:
XX = pos_star[:,:,0] # 99 x 1080
YY = pos_star[:,:,1] # 99 x 1080
ZZ = pos_star[:,:,2] # 99 x 1080
TT = np.tile(t_star.T, (1,N)) # 99 x 1080

UU = vel_star[:,:,0]
VV = vel_star[:,:,1]
WW = vel_star[:,:,2]

x = XX.flatten()[:,None] # TN x 1
y = YY.flatten()[:,None] # TN x 1
z = ZZ.flatten()[:,None] # TN x 1
t = TT.flatten()[:,None] # TN x 1

u = UU.flatten()[:,None] # TN x 1
v = VV.flatten()[:,None] # TN x 1
w = WW.flatten()[:,None] # TN x 1

In [61]:
print(UU.shape)

(99, 1080)


In [62]:
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
z_train = z[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]
w_train = w[idx,:]

# Training
model = PhysicsInformedNN(x_train, y_train, z_train, t_train, u_train, v_train, w_train, layers)
model.train(100000)     

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1

It: 0, Loss: 4.845e+05
It: 10, Loss: 4.684e+05
It: 20, Loss: 4.089e+05
It: 30, Loss: 3.111e+05
It: 40, Loss: 2.463e+05
It: 50, Loss: 1.952e+05
It: 60, Loss: 1.545e+05
It: 70, Loss: 1.269e+05
It: 80, Loss: 1.074e+05
It: 90, Loss: 9.281e+04
It: 100, Loss: 8.159e+04
It: 110, Loss: 7.314e+04
It: 120, Loss: 6.673e+04
It: 130, Loss: 6.197e+04
It: 140, Loss: 5.851e+04
It: 150, Loss: 5.603e+04
It: 160, Loss: 5.425e+04
It: 170, Loss: 5.300e+04
It: 180, Loss: 5.213e+04
It: 190, Loss: 5.152e+04
It: 200, Loss: 5.107e+04
It: 210, Loss: 5.072e+04
It: 220, Loss: 5.041e+04
It: 230, Loss: 5.014e+04
It: 240, Loss: 4.989e+04
It: 250, Loss: 4.964e+04
It: 260, Loss: 4.940e+04
It: 270, Loss: 4.916e+04
It: 280,

In [63]:
# Test Data
snap = np.array([50])
x_star = pos_star[snap,:,0:1][0]
y_star = pos_star[snap,:,1:2][0]
z_star = pos_star[snap,:,2:3][0]
t_star = TT[snap,:].T

u_star = vel_star[snap,:,0].T
v_star = vel_star[snap,:,1].T
w_star = vel_star[snap,:,2].T

# Prediction
u_pred, v_pred, w_pred, p_pred = model.predict(x_star, y_star, z_star, t_star)
# lambda_1_value = model.sess.run(model.lambda_1)
# lambda_2_value = model.sess.run(model.lambda_2)

# Error
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_w = np.linalg.norm(w_star-w_pred,2)/np.linalg.norm(w_star,2)

# error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
# error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100

print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error w: %e' % (error_w))    
# print('Error l1: %.5f%%' % (error_lambda_1))                             
# print('Error l2: %.5f%%' % (error_lambda_2))    

Error u: 9.998249e-01
Error v: 1.009381e+00
Error w: 1.002654e+00


In [74]:
# Test Data
snap = np.array([2])
x_star = pos_star[snap,:,0:1][0]
y_star = pos_star[snap,:,1:2][0]
z_star = pos_star[snap,:,2:3][0]
t_star = TT[snap,:].T

u_star = vel_star[snap,:,0].T
v_star = vel_star[snap,:,1].T
w_star = vel_star[snap,:,2].T

# Prediction
u_pred, v_pred, w_pred, p_pred = model.predict(x_star, y_star, z_star, t_star)
# lambda_1_value = model.sess.run(model.lambda_1)
# lambda_2_value = model.sess.run(model.lambda_2)

# Error
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_w = np.linalg.norm(w_star-w_pred,2)/np.linalg.norm(w_star,2)

# error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
# error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100

print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error w: %e' % (error_w)) 

Error u: inf
Error v: 1.941146e-02
Error w: inf


/root/anaconda3/envs/tfEnv/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in float_scalars
/root/anaconda3/envs/tfEnv/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in float_scalars


(1, 1080)


In [ ]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()

In [ ]:
# Plot Results
plot_solution(X_star, u_pred, 1)
plot_solution(X_star, v_pred, 2)